## Land use and cover changes under deforastation

### Amazon and Cerrado Biomes

In [9]:
# Importing Packeges

import rasterio
from rasterio.features import shapes
from rasterio.mask import mask as mask
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.features import shapes
from rasterio.plot import show 
from shapely.geometry import shape
import numpy as np
import fiona
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from osgeo import gdal
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.patches as mpatches
import seaborn as sns

print(f'Rasterio version: {rasterio.__version__}')
print(f'fiona version: {fiona.__version__}')

Rasterio version: 1.2.6
fiona version: 1.8.20


#### Reading paths and files

In [14]:
# Defining variables and paths: 
YEAR = 2020
ROI = 'Agricultural-Expansion'

ROI_SHAPE = "data/desmatamento_bioma_amz_PRODES_2020.shp"
ROI_LUC = f"data/LUC_mapbiomas-brazil-collection-7-matogrosso-2020-projetado-SIRGAS2000.tif"

export_tif_path_luc = f"data/image_{YEAR}_LUC_{ROI}.tif"

In [15]:
# Defining clip function

def cropa_tif_from_shp(shp_path, tif_path, export_tif_path):
    print(f'- Cropando {tif_path} com {shp_path}')
    with fiona.open(shp_path, "r") as shapefile:
        roi_shapes = [feature["geometry"] for feature in shapefile]
    
    tif_open = rasterio.open(tif_path)
    masked_tif, masked_tif_transform = rasterio.mask.mask(tif_open, roi_shapes, crop=True) 
    scar_meta = tif_open.meta
    
    scar_meta.update({ 
                 "height": masked_tif.shape[1],
                 "width": masked_tif.shape[2],
                 "transform": masked_tif_transform,
                 "nodata": 0})
    
    tif_result = np.where(masked_tif==256, 0, masked_tif)
    
    print(f'Salvando em {export_tif_path}\n')
    with rasterio.open(export_tif_path, "w", **scar_meta) as dest:
        dest.write(tif_result)

In [16]:
cropa_tif_from_shp(ROI_SHAPE, ROI_LUC, export_tif_path_luc)

- Cropando data/LUC_mapbiomas-brazil-collection-7-matogrosso-2020-projetado-SIRGAS2000.tif com data/desmatamento_bioma_amz_PRODES_2020.shp
Salvando em data/image_2020_LUC_Agricultural-Expansion.tif

